In [ ]:
#pip install selenium

In [37]:
import csv
import sys
import re
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException 

In [57]:
def check_element_exist(driver, selector, by = 'css'):
    try:
        if by == 'css':
            driver.find_element(By.CSS_SELECTOR, selector)
        elif by == 'class':
            driver.find_element(By.CLASS_NAME, selector)
    except NoSuchElementException:
        return False
    return True

In [39]:
def get_restaurant_list(url):
    driver = webdriver.Safari()
    driver.get(url)
    links = []
    page = 1
    page_link_count = 0
    selector = 'a.nav.next.rndBtn.ui_button.primary.taLnk'
    while check_element_exist(driver, 'a.nav.next.rndBtn.ui_button.primary.taLnk'):
        chunks = driver.find_elements(By.CSS_SELECTOR,'#component_2 > div > div > span')
        for c in chunks:
            if check_element_exist(c, 'span.NoCoR'):
                review_count = int(re.findall(r'\d+',c.find_element(By.CSS_SELECTOR,'span.NoCoR').text)[0])
                if (check_element_exist(c, 'div.bXDMz') == False) and (review_count >= 10):
                    restaurant = c.find_element(By.CSS_SELECTOR,'div.OhCyu > span > a')
                    links.append(restaurant.get_attribute('href'))
                    page_link_count += 1
        driver.find_element(By.CSS_SELECTOR, selector).send_keys("\n")
        print('page'+str(page)+': '+str(page_link_count)+'links')
        page += 1
        time.sleep(4)
    driver.close()
    return links

In [66]:
def get_restaurant_info(links, filename = 'restaurant.csv'):
    driver = webdriver.Safari()
    csv_file = open(filename, 'a', encoding="utf-8")
    csv_writer = csv.writer(csv_file)
    csv_writer.writerow(['restaurant_name', 'link', 
                         'price_class', 'price_range', 'category', 'michelin', 'ranking', 
                         'avg_rating', 'food_rating', 'service_rating', 'value_rating', 'atmosphere_rating',
                         'review_count', 'rate5_count', 'rate4_count', 'rate3_count', 'rate2_count', 'rate1_count'])
    restaurant_df = pd.DataFrame()
    for index, link in enumerate(links):
        time.sleep(2)
        driver.get(link)

        restaurant_name = driver.find_element(By.CSS_SELECTOR,'div.eTnlN._W.w.O > h1').text
        
        if check_element_exist(driver, 'cfvAV', by = 'class'):
            pri_ran = driver.find_element(By.CLASS_NAME,'cfvAV')
            price_range = [int(i) for i in re.findall(r'\d+', pri_ran.text)]
        elif check_element_exist(driver, 'bYIkW', by = 'class'):
            pri_ran = driver.find_element(By.CLASS_NAME,'bYIkW')
            price_range = [int(i) for i in re.findall(r'\d+', pri_ran.text)]
        else:
            price_range = []

        pri_cat = driver.find_elements(By.CSS_SELECTOR,'span.dyeJW.VRlVV > a')
        price_class = (pri_cat[0].text).count('$')
        if price_class == 5:
            price_class = 2.5
        category = [cat.text for cat in pri_cat[1:]]
        
        michelin = check_element_exist(driver, 'div.eabcK.Pc > div > div > div')
        ranking = int(re.findall(r'\d+', 
                    driver.find_element(By.CSS_SELECTOR,'div.fYCpi > b > span').text)[0])
        
        review_count = int(re.findall(r'\d+', 
                        driver.find_element(By.CSS_SELECTOR,'div.eEwDq > a').text)[0])
        avg_rating = int(re.findall(r'\d+', 
                        driver.find_element(By.CSS_SELECTOR,'span.fdsdx').text)[0])

        cat_ratings = driver.find_elements(By.CLASS_NAME,'cwxUN')
        if len(cat_ratings) == 4:
            food_element = cat_ratings[0].find_element(By.CSS_SELECTOR,"span")
            food_rating = int(re.findall(r'\d+', food_element.get_attribute('class'))[0])/10
            service_element = cat_ratings[1].find_element(By.CSS_SELECTOR,"span")
            service_rating = int(re.findall(r'\d+', service_element.get_attribute('class'))[0])/10
            value_element = cat_ratings[2].find_element(By.CSS_SELECTOR,"span")
            value_rating = int(re.findall(r'\d+', value_element.get_attribute('class'))[0])/10
            atmosphere_element = cat_ratings[2].find_element(By.CSS_SELECTOR,"span")
            atmosphere_rating = int(re.findall(r'\d+', atmosphere_element.get_attribute('class'))[0])/10
        else:
            food_rating, service_rating, value_rating, atmosphere_rating = 'NA', 'NA', 'NA', 'NA'

        rate_distri = driver.find_elements(By.CSS_SELECTOR,'span.row_num.is-shown-at-tablet')
        rate5_count = int(rate_distri[0].text)
        rate4_count = int(rate_distri[1].text)
        rate3_count = int(rate_distri[2].text)
        rate2_count = int(rate_distri[3].text)
        rate1_count = int(rate_distri[4].text)
        
        csv_writer.writerow([restaurant_name, link, 
                             price_class, price_range, category, michelin, ranking, 
                             avg_rating, food_rating, service_rating, value_rating, atmosphere_rating,
                             review_count, rate5_count, rate4_count, rate3_count, rate2_count, rate1_count])
        print(str(index), restaurant_name)
    driver.close()    

In [41]:
def check_next_page(driver, selector):
    if check_element_exist(driver, selector):
        next_page_url = driver.find_element(By.CSS_SELECTOR, selector)\
                        .get_attribute('href')
        if next_page_url != '':
            if next_page_url != None:
                return 'Next'
            else:
                return 'Last'
    return 'None'

In [42]:
def get_restaurant_reviews(links, filename = 'reviews.csv'):
    driver = webdriver.Safari()
    csv_file = open(filename, 'a', encoding="utf-8")
    csv_writer = csv.writer(csv_file)
    csv_writer.writerow(['restaurant_name', 'link', 'date', 'rating', 'title', 'review'])
    for index, link in enumerate(links):
        driver.get(link)
        restaurant_name = driver.find_element(By.CSS_SELECTOR,'div.eTnlN._W.w.O > h1').text
        page = 0
        review_count = 0
        next_page = link
        while check_next_page(driver, 'a.nav.next.ui_button.primary') != 'None':
            time.sleep(1)
            driver.get(next_page)
            reviews = driver.find_elements(By.XPATH,".//div[@class='review-container']")
            if check_element_exist(driver, 'span.taLnk.ulBlueLinks'):
                driver.find_element(By.XPATH,"//span[@class='taLnk ulBlueLinks']").click()
                time.sleep(2)
            for review in reviews:
                title = review.find_element(By.CSS_SELECTOR,'div.quote').text
                date = review.find_element(By.CSS_SELECTOR, 'span.ratingDate').get_attribute("title")
                rating = int(review.find_element(By.XPATH,".//span[contains(@class, 'ui_bubble_rating bubble_')]")\
                             .get_attribute("class").split("_")[3])//10
                review = review.find_element(By.CSS_SELECTOR,"div.prw_rup.prw_reviews_text_summary_hsx > div > p")\
                             .text.replace("\n", " ")
                
                csv_writer.writerow([restaurant_name, link, date, rating, title, review])
                review_count += 1
            print(str(index)+' '+restaurant_name+': '+str(review_count)+'reviews')
            if check_next_page(driver, 'a.nav.next.ui_button.primary') == 'Last':
                break
            page += 10
            next_page = re.sub(r'(Reviews-)', r'\1or'+str(page)+'-', link)
    driver.close()

In [45]:
#example: Chinatown
url = "https://www.tripadvisor.com/Restaurants-g35805-zfn7778526-Chicago_Illinois.html"
links = get_restaurant_list(url)

page1: 30links
page2: 49links
page3: 50links


In [70]:
get_restaurant_info(links[38:], 'restaurants_chinatown.csv')
restaurants_df = pd.read_csv('restaurants_chinatown.csv')
restaurants_df.head()

0 Great Wall Restaurant
1 Yan Bang Cai
2 Xi'an
3 Little Lamb
4 Cafe Hoang
5 Mayflower Chinese Restaurant
6 Bonchon
7 Sweet Station
8 Lao Beijing
9 Lee Wing Wah
10 Original Triple Crown Restaurant
11 Lao Yunnan


,restaurant_name,link,price_class,price_range,category,michelin,ranking,avg_rating,food_rating,service_rating,value_rating,atmosphere_rating,review_count,rate5_count,rate4_count,rate3_count,rate2_count,rate1_count
0,MingHin Cuisine,https://www.tripadvisor.com/Restaurant_Review-...,2.5,"[10, 18]","['Chinese', 'Asian', 'Cantonese']",True,181,4,4.5,4.0,4.0,4.0,443,183,150,56,15,7
1,Connie's Pizza,https://www.tripadvisor.com/Restaurant_Review-...,2.5,"[15, 30]","['Italian', 'Pizza', 'Vegetarian Friendly']",False,62,4,4.5,4.5,4.5,4.5,177,96,55,12,7,2
2,Joy Yee Noodles,https://www.tripadvisor.com/Restaurant_Review-...,2.5,[],"['Chinese', 'Asian', 'Hong Kong']",False,329,4,4.5,4.0,4.5,4.5,208,97,64,22,8,6
3,Triple Crown Restaurant,https://www.tripadvisor.com/Restaurant_Review-...,2.5,"[10, 30]","['Chinese', 'Asian', 'Cantonese']",False,345,4,4.0,3.5,4.0,4.0,245,85,80,30,12,8
4,Moon Palace Restaurant,https://www.tripadvisor.com/Restaurant_Review-...,2.5,"[10, 30]","['Chinese', 'Asian', 'Shanghai']",False,9,4,4.5,4.5,4.5,4.5,115,56,37,12,0,2


In [71]:
get_restaurant_reviews(links, 'reviews_chinatown.csv')
reviews_df = pd.read_csv('reviews_chinatown.csv')
reviews_df.head()

0 MingHin Cuisine: 10reviews
0 MingHin Cuisine: 20reviews
0 MingHin Cuisine: 30reviews
0 MingHin Cuisine: 40reviews
0 MingHin Cuisine: 50reviews
0 MingHin Cuisine: 60reviews
0 MingHin Cuisine: 70reviews
0 MingHin Cuisine: 80reviews
0 MingHin Cuisine: 90reviews
0 MingHin Cuisine: 100reviews
0 MingHin Cuisine: 110reviews
0 MingHin Cuisine: 120reviews
0 MingHin Cuisine: 130reviews
0 MingHin Cuisine: 140reviews
0 MingHin Cuisine: 150reviews
0 MingHin Cuisine: 160reviews
0 MingHin Cuisine: 170reviews
0 MingHin Cuisine: 180reviews
0 MingHin Cuisine: 190reviews
0 MingHin Cuisine: 200reviews
0 MingHin Cuisine: 210reviews
0 MingHin Cuisine: 220reviews
0 MingHin Cuisine: 230reviews
0 MingHin Cuisine: 240reviews
0 MingHin Cuisine: 250reviews
0 MingHin Cuisine: 260reviews
0 MingHin Cuisine: 270reviews
0 MingHin Cuisine: 280reviews
0 MingHin Cuisine: 290reviews
0 MingHin Cuisine: 300reviews
0 MingHin Cuisine: 310reviews
0 MingHin Cuisine: 320reviews
0 MingHin Cuisine: 330reviews
0 MingHin Cuisine: 

25 The Noodle: 20reviews
25 The Noodle: 30reviews
25 The Noodle: 40reviews
25 The Noodle: 42reviews
26 Golden Bull Restaurant: 10reviews
26 Golden Bull Restaurant: 20reviews
26 Golden Bull Restaurant: 24reviews
27 My Place: 10reviews
27 My Place: 16reviews
28 Sze Chuan Cuisine: 10reviews
28 Sze Chuan Cuisine: 20reviews
28 Sze Chuan Cuisine: 28reviews
29 Saint's Alp Teahouse: 10reviews
29 Saint's Alp Teahouse: 20reviews
29 Saint's Alp Teahouse: 24reviews
30 Three Happiness Restaurant: 10reviews
30 Three Happiness Restaurant: 20reviews
30 Three Happiness Restaurant: 30reviews
30 Three Happiness Restaurant: 40reviews
30 Three Happiness Restaurant: 50reviews
30 Three Happiness Restaurant: 56reviews
31 Saint Anna Bakery & Cafe: 10reviews
31 Saint Anna Bakery & Cafe: 20reviews
31 Saint Anna Bakery & Cafe: 21reviews
32 Ahjoomah's Apron: 10reviews
32 Ahjoomah's Apron: 20reviews
32 Ahjoomah's Apron: 29reviews
33 Seven Treasures Cantonese Restaurant: 10reviews
33 Seven Treasures Cantonese Restau

,restaurant_name,link,date,rating,title,review
0,MingHin Cuisine,https://www.tripadvisor.com/Restaurant_Review-...,"December 29, 2021",2,Disappointing Dimsum,We chose Minghin for dimsum because reviews we...
1,MingHin Cuisine,https://www.tripadvisor.com/Restaurant_Review-...,"November 20, 2021",5,Delicious and filling,Hot tea is always the way to start an evening ...
2,MingHin Cuisine,https://www.tripadvisor.com/Restaurant_Review-...,"October 24, 2021",3,Order pizza,Cuisine? $300 of food picked up for a family g...
3,MingHin Cuisine,https://www.tripadvisor.com/Restaurant_Review-...,"September 19, 2021",5,favorite dimsum!!,came here with family and some big expectation...
4,MingHin Cuisine,https://www.tripadvisor.com/Restaurant_Review-...,"August 30, 2021",4,Good dim sum and the restaurant was not a hole...,Highly recommend the dim sum. Comes out nice a...
